In [1]:
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import pandas as pd

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://stats.nba.com/teams/traditional'
driver.get(url)


# Click on playoff stats

driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[2]').click()
time.sleep(2)
driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[3]').click()
time.sleep(2)

num_js_seasons = 24 #ON NBA.com the team playoff win stats go back to 1997

df = pd.DataFrame(columns = ['Season', 'Team', 'gp', 'W', 'L'])


for season in range(2, num_js_seasons):
    
    #Select new season to scrape from
    season_path = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option['+str(season)+']'
    driver.find_element_by_xpath(season_path).click()
    
    
    # I have added sleep to give chromedriver time to load
    time.sleep(2)
    
    #Make sure we're still selecting playoff
    driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[3]').click()
    time.sleep(2)
    
    # Get the table
    table = driver.find_element_by_class_name('nba-stat-table__overflow')
    
    
    # This code parses the table
    team_ids = []
    team_names = []
    team_stats = []

    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id == 0:
            column_names = lines.split(' ')[1:]
        else:
            if line_id % 3 == 1:
                team_ids.append(lines)
            if line_id % 3 == 2:
                team_names.append(lines)
            if line_id % 3 == 0:
                team_stats.append( [float(i) for i in lines.split(' ')] )
                
    #Make pandas dataframe for this season
                
    playoff_wins = pd.DataFrame({'Season' : 2020-season,
                             'Team': team_names,
                       'gp': [i[0] for i in team_stats],
                       'W': [i[1] for i in team_stats],
                       'L': [i[2] for i in team_stats]
                       }
                     )
    
    #Append current season to dataframe
    
    df = df.append(playoff_wins)
    
#     print(df.tail())


Looking for [chromedriver 81.0.4044.69 mac64] driver in cache 
File found in cache by path [/Users/holden/.wdm/drivers/chromedriver/81.0.4044.69/mac64/chromedriver]


In [5]:
df.head()

,Season,Team,gp,W,L
0,2018,Milwaukee Bucks,15.0,10.0,5.0
1,2018,Toronto Raptors,24.0,16.0,8.0
2,2018,Golden State Warriors,22.0,14.0,8.0
3,2018,Philadelphia 76ers,12.0,7.0,5.0
4,2018,Boston Celtics,9.0,5.0,4.0


In [6]:
df.tail()

,Season,Team,gp,W,L
11,1997,Atlanta Hawks,4.0,1.0,3.0
12,1997,Cleveland Cavaliers,4.0,1.0,3.0
13,1997,Phoenix Suns,4.0,1.0,3.0
14,1997,Portland Trail Blazers,4.0,1.0,3.0
15,1997,New Jersey Nets,3.0,0.0,3.0


In [7]:
df.to_csv('playoff_wins.csv')